In [1]:
# Import all necessery python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import re
import requests as req
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import seaborn as sns

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, RidgeCV, Ridge, lars_path
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

from IPython.display import Image

import pickle

import folium

import geojson

import geopy
import geopy.distance

from IPython.display import display

In [2]:
def loadpickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [5]:
df = loadpickle('./data/dataframe.pkl')

# Add data

In [62]:
def extendColumnWithData(id):
    page = loadpickle('./data/details/'+ id + '.pkl')
    soup_ingatlan = BeautifulSoup(page.content,"lxml")
    subtype = soup_ingatlan.find('div', {'class':'listing-subtype'}).text
    description = soup_ingatlan.find('div', {'class':'long-description'}).text
    df.loc[df['ID']==id, 'SUBTYPE'] = subtype
    df.loc[df['ID']==id, 'DESCRIPTION'] = description

In [6]:
df['SUBTYPE'].unique()

array(['Eladó tégla építésű lakás', 'Eladó panel lakás',
       'Tégla építésű lakás bérleti joga átadó',
       'Új építésű lakás lakóparkban', 'Eladó csúszózsalus lakás',
       'Panel lakás bérleti joga átadó'], dtype=object)

In [7]:
df_work = df.copy()

In [8]:
df_work.head()

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,Balcony,ORIENTATION,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,SUBTYPE,DESCRIPTION
0,23545892,39 m²,"31,9 millió Ft",1,földszint,"7. kerület, Dob utca 50.",közepes állapotú,1950 előtt,2,nincs megadva,NaN,dél,utcai,False,"utca, közterület - fizetős övezet",7,47.501126,19.065041,Eladó tégla építésű lakás,"A VII. ker. Dob utcában, a körúton belül, egy..."
1,23652121,79 m²,"65,9 millió Ft",3,4,"6. kerület, Belső-Terézváros",felújított,1950 előtt,nincs megadva,nincs megadva,NaN,nincs megadva,nincs megadva,False,nincs megadva,6,47.509494,19.065323,Eladó tégla építésű lakás,"VI. ker. Zichy Jenő utcában eladó, egy 79 nm-..."
2,23662271,75 m²,89 millió Ft,2 + 1 fél,1,"7. kerület, Belső-Erzsébetváros",újszerű,1950 előtt,4,van,NaN,délnyugat,utcai,False,nincs megadva,7,47.502627,19.077243,Eladó tégla építésű lakás,"Belváros forgatagában, a Zsinagóga közelében,..."
3,23662566,50 m²,"22,9 millió Ft",2,4,"3. kerület, Bécsi út",felújítandó,1950-1980 között,10,van,NaN,nyugat,utcai,False,"utca, közterület - fizetős övezet",3,47.532673,19.036491,Eladó panel lakás,"Óbuda lakótelepen, a Bécsi úton 2 szobás laká..."
4,23688806,32 m²,"12,5 millió Ft",1,3,"9. kerület, Haller utca",jó állapotú,NaN,3,nincs,NaN,nincs megadva,nincs megadva,False,"utca, közterület",9,47.476099,19.077198,Tégla építésű lakás bérleti joga átadó,"Budapest , IX. kerületi 32 nm-es ,galériázott..."


In [9]:
df_work['SIZE'] = df['SIZE'].str[:-3].str.replace(" ","").astype('int')

In [10]:
df_work['SIZE']

0        39
1        79
2        75
3        50
4        32
         ..
18556    24
18557    90
18558    68
18559    84
18560    30
Name: SIZE, Length: 18561, dtype: int32

In [11]:
milliard_df = df_work[(df_work['PRICE'].str.endswith('milliárd Ft', na=False))]
millio_df = df_work[(df_work['PRICE'].str.endswith('millió Ft', na=False))]

In [12]:
df_work.loc[df_work['PRICE'].str.endswith('milliárd Ft', na=False), 'PRICE'] = milliard_df['PRICE'].str.split().str[0].str.replace(',', '.').astype('float') * 1000
df_work.loc[df_work['PRICE'].str.endswith('millió Ft', na=False), 'PRICE'] = millio_df['PRICE'].str.split().str[0].str.replace(',', '.').astype('float')

In [13]:
df_work['rooms'] = df_work['ROOMS']
df_work['half_rooms'] = 0

In [14]:
df_work[['rooms', 'half_rooms']] = df_work['ROOMS'].str[:-4].str.replace(" ","").str.split('\+',expand=True)

In [15]:
df_work.loc[~(df_work['ROOMS'].str.contains('fél', na=False)), 'rooms'] = df_work[~(df_work['ROOMS'].str.contains('\+', na=False))]['ROOMS']
#df_work['rooms'] = df_work['ROOMS']

In [16]:
df_work['half_rooms'].fillna(value=0, inplace=True)

In [17]:
df_work['rooms'] = df_work['rooms'].str.replace(" ","").astype('int')

In [18]:
df_work[df_work['rooms']>10 ]

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,...,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,SUBTYPE,DESCRIPTION,rooms,half_rooms
97,25441982,1020,1200,52,nincs megadva,"7. kerület, Király utca",nincs megadva,NaN,nincs megadva,nincs megadva,...,utcai,False,teremgarázs hely - benne van az árban,7,47.506331,19.070639,Eladó tégla építésű lakás,"VII. kerület, belváros, Király utcánál, Nagym...",52,0
667,27935639,140,89.9,11,földszint,"6. kerület, Külső-Terézváros",jó állapotú,NaN,nincs megadva,van,...,utcai,False,nincs megadva,6,47.509494,19.065323,Eladó tégla építésű lakás,"Eladásra kínálunk a 6. kerületben, a Nyugati ...",11,0
1160,28706021,351,239.9,12,1,"12. kerület, Városmajor utca",jó állapotú,1950 előtt,3,nincs,...,kertre néző,False,"utca, közterület",12,47.504458,19.019731,Eladó tégla építésű lakás,"A két legnagyobb egymás fölötti, össze is nyi...",12,0
1419,28946876,825,580,30,4,"7. kerület, Damjanich utca",újszerű,2001-2010 között,4,van,...,utcai,False,"utca, közterület",7,47.506144,19.074918,Eladó tégla építésű lakás,Múzeumnegyed szomszédságában Damjanich utcáná...,30,0
2085,29325197,345,450,12,3,"7. kerület, Belső-Erzsébetváros",felújított,2019,nincs megadva,van,...,nincs megadva,False,nincs megadva,7,47.502627,19.077243,Eladó tégla építésű lakás,Egy liftes klasszikus épület harmadik emeleté...,12,0
5708,30386137,360,450,12,2,"7. kerület, Dohány utca",nincs megadva,NaN,5,van,...,utcai,False,nincs megadva,7,47.500448,19.073166,Eladó tégla építésű lakás,Eladó 7. kerület Dohány utca Blaha Lujza térn...,12,0
6269,30470449,700,699,40,földszint,"5. kerület, Váci utca",közepes állapotú,1950-1980 között,3,nincs megadva,...,nincs megadva,False,nincs megadva,5,47.493201,19.052852,Eladó tégla építésű lakás,"EGYEDI LEHETŐSÉG!Váci utca csendes, Vásárcsar...",40,0
10541,30933370,2222222,222220,222,nincs megadva,"7. kerület, Rákóczi út",nincs megadva,NaN,nincs megadva,nincs megadva,...,nincs megadva,False,nincs megadva,7,47.499044,19.077575,Eladó tégla építésű lakás,"Ez egy automatikusan, vagy kollégáink által g...",222,0
11055,30969868,835,799,14,nincs megadva,"2. kerület, Rózsadomb",nincs megadva,1950 előtt,nincs megadva,van,...,nincs megadva,False,nincs megadva,2,47.517329,19.028288,Eladó tégla építésű lakás,Panorámás villa eladók!,14,0
12787,31087012,222,999,11 + 18 fél,nincs megadva,"1. kerület, Krisztina tér",nincs megadva,NaN,nincs megadva,nincs megadva,...,nincs megadva,False,nincs megadva,1,47.496863,19.031734,Eladó panel lakás,teszt!,11,18


In [19]:
df_work[(df_work['ROOMS'].str.contains('\+', na=False))]

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,...,VIEW,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,SUBTYPE,DESCRIPTION,rooms,half_rooms
2,23662271,75,89,2 + 1 fél,1,"7. kerület, Belső-Erzsébetváros",újszerű,1950 előtt,4,van,...,utcai,False,nincs megadva,7,47.502627,19.077243,Eladó tégla építésű lakás,"Belváros forgatagában, a Zsinagóga közelében,...",2,1
8,23792850,126,169.9,3 + 1 fél,4,"5. kerület, Zrínyi utca",felújítandó,NaN,6,van,...,panorámás,False,"utca, közterület",5,47.500191,19.048434,Eladó tégla építésű lakás,Kivételes lehetőség a Bazilika szomszédságába...,3,1
16,23936899,65,8.65,1 + 2 fél,földszint,"9. kerület, Üllői út",közepes állapotú,1950 előtt,3,nincs,...,utcai,False,"utca, közterület - fizetős övezet",9,47.473919,19.103220,Tégla építésű lakás bérleti joga átadó,Önkormányzati bérlakás bérleti joga értékegye...,1,2
21,24171833,85,106.29,1 + 1 fél,nincs megadva,"2. kerület, Budakeszi út",újszerű,2001-2010 között,6,van,...,panorámás,False,teremgarázs hely - benne van az árban,2,47.517581,18.995488,Eladó tégla építésű lakás,Budapest II. ker. Hárshegyen a luxuskivitelű ...,1,1
32,24406258,69,69.9,2 + 1 fél,2,"6. kerület, Zichy Jenő utca",felújított,2019,5,van,...,utcai,False,"utca, közterület",6,47.506963,19.060848,Eladó tégla építésű lakás,"Akar Ön a város szívében, egy DÍJNYERTES TERV...",2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18547,31370110,58,37.9,2 + 1 fél,1,"22. kerület, Tűzliliom utca 29.",jó állapotú,1981-2000 között,4,nincs,...,nincs megadva,False,"utca, közterület - ingyenes",22,47.407227,19.007124,Eladó panel lakás,"Nagy lett immár a családunk, kinőttük a lakás...",2,1
18554,31370269,42,42.9,1 + 1 fél,földszint,"11. kerület, Stoczek utca",felújított,1950 előtt,5,nincs,...,utcai,False,"utca, közterület - fizetős övezet",11,47.477558,19.055940,Eladó tégla építésű lakás,ELADÓ 42 M2-ES DESIGN LAKÁS A XI. KERÜLETI MŰ...,1,1
18555,31370344,132,120,4 + 1 fél,2,"11. kerület, Karinthy Frigyes út",jó állapotú,1950-1980 között,4,van,...,utcai,False,"utca, közterület - fizetős övezet",11,47.477591,19.048362,Eladó tégla építésű lakás,Tulajdonostó eladó. Az ingatlan helyileg Buda...,4,1
18557,31370386,90,69.9,1 + 3 fél,3,"8. kerület, Üllői út 64.",felújított,1950 előtt,3,nincs,...,udvari,False,udvari beálló - bérelhető,8,47.484385,19.075654,Eladó tégla építésű lakás,A környék legszebb épületében teljeskörűen fe...,1,3


In [20]:
df_work['FLOOR'].unique()

array(['földszint', '4', '1', '3', 'nincs megadva', '2', '5', '7', '8',
       'félemelet', 'szuterén', '6', '9', '10', '10 felett'], dtype=object)

In [21]:
df_work.loc[(df_work['FLOOR']=='földszint'), 'FLOOR'] = 0
df_work.loc[(df_work['FLOOR']=='félemelet'), 'FLOOR'] = 1
df_work.loc[(df_work['FLOOR']=='szuterén'), 'FLOOR'] = -1
df_work.loc[(df_work['FLOOR']=='10 felett'), 'FLOOR'] = 11
df_work.loc[(df_work['FLOOR']=='nincs megadva'), 'FLOOR'] = 0
df_work['FLOOR'] = df_work['FLOOR'].astype('int')

In [22]:
df_work['QUALITY'].unique() 

array(['közepes állapotú', 'felújított', 'újszerű', 'felújítandó',
       'jó állapotú', 'új építésű', 'nincs megadva', 'befejezetlen'],
      dtype=object)

In [23]:
df_work['YEAR_BUILT'].unique() 

array(['1950 előtt', '1950-1980 között', nan, '2019', '2017', '2020',
       '2001-2010 között', '2015', '2018', '2012', '1981-2000 között',
       '2021', '2011', '2016', '2022', '2013', '2014'], dtype=object)

In [24]:
df_work['PRICE_PER_METER'] = df_work['PRICE'] / df_work['SIZE']

In [25]:
df_work.head(20)

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,...,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,SUBTYPE,DESCRIPTION,rooms,half_rooms,PRICE_PER_METER
0,23545892,39,31.9,1,0,"7. kerület, Dob utca 50.",közepes állapotú,1950 előtt,2,nincs megadva,...,False,"utca, közterület - fizetős övezet",7,47.501126,19.065041,Eladó tégla építésű lakás,"A VII. ker. Dob utcában, a körúton belül, egy...",1,0,0.817949
1,23652121,79,65.9,3,4,"6. kerület, Belső-Terézváros",felújított,1950 előtt,nincs megadva,nincs megadva,...,False,nincs megadva,6,47.509494,19.065323,Eladó tégla építésű lakás,"VI. ker. Zichy Jenő utcában eladó, egy 79 nm-...",3,0,0.834177
2,23662271,75,89,2 + 1 fél,1,"7. kerület, Belső-Erzsébetváros",újszerű,1950 előtt,4,van,...,False,nincs megadva,7,47.502627,19.077243,Eladó tégla építésű lakás,"Belváros forgatagában, a Zsinagóga közelében,...",2,1,1.18667
3,23662566,50,22.9,2,4,"3. kerület, Bécsi út",felújítandó,1950-1980 között,10,van,...,False,"utca, közterület - fizetős övezet",3,47.532673,19.036491,Eladó panel lakás,"Óbuda lakótelepen, a Bécsi úton 2 szobás laká...",2,0,0.458
4,23688806,32,12.5,1,3,"9. kerület, Haller utca",jó állapotú,NaN,3,nincs,...,False,"utca, közterület",9,47.476099,19.077198,Tégla építésű lakás bérleti joga átadó,"Budapest , IX. kerületi 32 nm-es ,galériázott...",1,0,0.390625
5,23722424,75,64.7,3,4,"13. kerület, The Gardens Budapest",új építésű,2019,5,van,...,False,teremgarázs hely - megvásárolható,13,47.533498,19.070615,Új építésű lakás lakóparkban,"- Kockázatmentes fizetési ütemezés, két részl...",3,0,0.862667
6,23745768,77,95,2,0,"6. kerület, Dózsa György út 102.",nincs megadva,NaN,nincs megadva,nincs megadva,...,False,nincs megadva,6,47.515406,19.075342,Eladó tégla építésű lakás,VILLA LAKÁS ELADÓ!! VILLA LAKAS ELADO !! ELAD...,2,0,1.23377
7,23776860,45,42.9,2,1,"8. kerület, Víg utca",új építésű,2019,6,van,...,False,teremgarázs hely - megvásárolható,8,47.493491,19.073544,Eladó tégla építésű lakás,MÁR CSAK 1 SZABAD LAKÁS - NE MARADJON LE!JOGE...,2,0,0.953333
8,23792850,126,169.9,3 + 1 fél,4,"5. kerület, Zrínyi utca",felújítandó,NaN,6,van,...,False,"utca, közterület",5,47.500191,19.048434,Eladó tégla építésű lakás,Kivételes lehetőség a Bazilika szomszédságába...,3,1,1.34841
9,23846400,62,75,2,0,"5. kerület, Károly körút 26.",nincs megadva,NaN,nincs megadva,nincs megadva,...,False,nincs megadva,5,47.496192,19.057291,Eladó tégla építésű lakás,"A belváros szívében, a Ritz Carlton Budapest ...",2,0,1.20968


In [26]:
df_work.sort_values(by=['PRICE_PER_METER'])

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,...,METRO_NEARBY,PARKING,DISTRICT,LATITUDE,LONGITUDE,SUBTYPE,DESCRIPTION,rooms,half_rooms,PRICE_PER_METER
10541,30933370,2222222,222220,222,0,"7. kerület, Rákóczi út",nincs megadva,NaN,nincs megadva,nincs megadva,...,False,nincs megadva,7,47.499044,19.077575,Eladó tégla építésű lakás,"Ez egy automatikusan, vagy kollégáink által g...",222,0,0.099999
10591,30937240,51,5.5,1 + 2 fél,5,"3. kerület, Hollós Korvin Lajos utca",jó állapotú,NaN,nincs megadva,van,...,False,nincs megadva,3,47.597314,19.050965,Panel lakás bérleti joga átadó,Önkormányzati lakás bérleteBékásmegyeri határ...,1,2,0.107843
16,23936899,65,8.65,1 + 2 fél,0,"9. kerület, Üllői út",közepes állapotú,1950 előtt,3,nincs,...,False,"utca, közterület - fizetős övezet",9,47.473919,19.103220,Tégla építésű lakás bérleti joga átadó,Önkormányzati bérlakás bérleti joga értékegye...,1,2,0.133077
10657,30942235,41,5.5,1 + 1 fél,0,"14. kerület, Kacsóh Pongrác út",nincs megadva,NaN,nincs megadva,nincs megadva,...,False,nincs megadva,14,47.521993,19.092631,Eladó tégla építésű lakás,Zuglóban a Kacsóh Pongrác úton parkra néző 41...,1,1,0.134146
3401,29829311,54,7.9,2,1,"22. kerület, Gyár utca",felújított,1950-1980 között,2,nincs,...,False,"utca, közterület - ingyenes",22,47.412131,19.038537,Tégla építésű lakás bérleti joga átadó,"Budapest XXII. kerületében, a Gyár utcában el...",2,0,0.146296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,27943529,102,400,2 + 1 fél,4,"5. kerület, Kossuth Lajos tér",felújított,1950 előtt,6,van,...,False,"utca, közterület - fizetős övezet",5,47.505720,19.044981,Eladó tégla építésű lakás,PÁRATLAN POMPA A DUNA MENTÉN!Budapest legszeb...,2,1,3.92157
10912,30959707,9,36,1,0,"5. kerület, Váci utca",nincs megadva,NaN,nincs megadva,nincs megadva,...,False,nincs megadva,5,47.493201,19.052852,Eladó tégla építésű lakás,Eladó Budapest V. kerület Váci utcai II.emele...,1,0,4
691,28071215,100,400,2 + 1 fél,4,"5. kerület, Kossuth Lajos tér",újszerű,1950 előtt,6,van,...,False,"utca, közterület - fizetős övezet",5,47.505720,19.044981,Eladó tégla építésű lakás,Az V. kerületben a Kossuth Téren eladó egy 10...,2,1,4
12787,31087012,222,999,11 + 18 fél,0,"1. kerület, Krisztina tér",nincs megadva,NaN,nincs megadva,nincs megadva,...,False,nincs megadva,1,47.496863,19.031734,Eladó panel lakás,teszt!,11,18,4.5


## BKK

In [40]:
stops = pd.read_csv('./data/bkk/stops.txt')
stop_times = pd.read_csv('./data/bkk/stop_times.txt')
trips = pd.read_csv('./data/bkk/trips.txt')
routes = pd.read_csv('./data/bkk/routes.txt')

In [41]:
route_to_trip = trips[['route_id', 'trip_id']]
stop_to_trip = stop_times[['trip_id', 'stop_id']]

In [42]:
stops_filtered = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]

In [43]:
all_stops = pd.merge(stop_to_trip.join(route_to_trip.set_index('trip_id'), on='trip_id'), stops_filtered, how='left', on='stop_id')

In [44]:
all_stops = all_stops.drop(columns=['trip_id', 'stop_id']).drop_duplicates()

In [49]:
all_stops = all_stops.join(routes[['route_id', 'route_type']].set_index('route_id'), on='route_id')

In [51]:
all_stops[all_stops['route_type'] == 1]

,route_id,stop_name,stop_lat,stop_lon,route_type
0,5200,Déli pályaudvar,47.500450,19.024604,1.0
1,5200,Széll Kálmán tér,47.507533,19.026097,1.0
2,5200,Batthyány tér,47.506826,19.038103,1.0
3,5200,Kossuth Lajos tér,47.505511,19.045769,1.0
4,5200,Deák Ferenc tér,47.497593,19.053857,1.0
...,...,...,...,...,...
1200201,5300,Deák Ferenc tér,47.497591,19.054525,1.0
1200202,5300,Kálvin tér,47.489490,19.061920,1.0
1200203,5300,Corvin-negyed,47.484862,19.072242,1.0
1200204,5300,Semmelweis Klinikák,47.482872,19.078425,1.0


In [53]:
all_stops[all_stops['route_type'] == 0]

,route_id,stop_name,stop_lat,stop_lon,route_type
48194,3480,Deák Ferenc tér M,47.497413,19.055158,0.0
48195,3480,Astoria M,47.494561,19.059696,0.0
48196,3480,Kálvin tér M,47.489219,19.061248,0.0
48197,3480,Fővám tér M,47.486916,19.057240,0.0
48198,3480,Szent Gellért tér - Műegyetem M,47.483337,19.053086,0.0
...,...,...,...,...,...
1572243,3040,Asztalos Sándor út,47.490015,19.105515,0.0
1572244,3040,Hidegkuti Nándor Stadion,47.490040,19.108249,0.0
1945544,3248,Magdolna utca,47.492873,19.087298,0.0
1945545,3248,Dologház utca,47.496364,19.083399,0.0


In [100]:
all_stops[all_stops['route_type'] == 3]

,route_id,stop_name,stop_lat,stop_lon,route_type
32934,1570,"Solymár, Kerekhegy",47.581449,18.926420,3.0
32935,1570,Muflon utca,47.570670,18.931469,3.0
32936,1570,"Budaliget, Géza fejedelem útja",47.567607,18.936286,3.0
32937,1570,Honfoglalás utca,47.568852,18.940151,3.0
32938,1570,Köztársaság utca,47.566417,18.942349,3.0
...,...,...,...,...,...
2436016,1220,"Megyer, Szondi utca",47.586247,19.095104,3.0
2436017,1220,Óceán-árok utca,47.590345,19.101781,3.0
2436018,1220,Íves út,47.592824,19.107979,3.0
2436019,1220,Homoktövis utca,47.594692,19.113273,3.0


In [131]:
routes['route_type'].unique()

array([  3,   0, 800,   1, 109,   4], dtype=int64)

In [81]:
tram_coord = all_stops[all_stops['route_type'] == 0][['stop_lat', 'stop_lon']]

In [82]:
tram_coord

,stop_lat,stop_lon
48194,47.497413,19.055158
48195,47.494561,19.059696
48196,47.489219,19.061248
48197,47.486916,19.057240
48198,47.483337,19.053086
...,...,...
1572243,47.490015,19.105515
1572244,47.490040,19.108249
1945544,47.492873,19.087298
1945545,47.496364,19.083399


In [91]:
tram_coord_list = [geopy.Point(x) for x in tram_coord.to_numpy()]

In [97]:
def filterForCloseStops(coord_list, treshold):
    processed = list()
    processed.append(coord_list[0])
    for coord in coord_list:
        meter = min([geopy.distance.distance(coord, m_coord).m for m_coord in processed])
        if meter > 50:
            processed.append(coord)
    return processed

In [93]:
len(processed)

460

In [94]:
metro_coord = all_stops[all_stops['route_type'] == 1][['stop_lat', 'stop_lon']]


In [98]:
metro_coord_list = filterForCloseStops([geopy.Point(x) for x in metro_coord.to_numpy()], 50)

In [102]:
def enrichDataFrameWithTransportData(mydf, coord_list, new_column_name, treshold):
    cnt = 0
    for i in mydf['ID']:
        coord = geopy.Point(mydf[mydf['ID']==i][['LATITUDE', 'LONGITUDE']].to_numpy())
        meter = min([geopy.distance.distance(coord, m_coord).m for m_coord in coord_list])
        if meter < treshold:
            mydf.loc[mydf['ID']==i, new_column_name] = True
        cnt +=1
        if cnt%100 == 0:
            print(cnt)

In [103]:
enrichDataFrameWithTransportData(df_work, metro_coord_list, 'METRO_NEARBY', 500)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [105]:
tram_coord_list = filterForCloseStops([geopy.Point(x) for x in tram_coord.to_numpy()], 50)

In [106]:
enrichDataFrameWithTransportData(df_work, tram_coord_list, 'TRAM_NEARBY', 250)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [109]:
df_work[['TRAM_NEARBY']] = df_work[['TRAM_NEARBY']].fillna(value=False)

In [113]:
df_work[~(df_work['METRO_NEARBY']) & (df_work['TRAM_NEARBY'])]

,ID,SIZE,PRICE,ROOMS,FLOOR,ADDRESS,QUALITY,YEAR_BUILT,FLOORS_IN_BUILDING,LIFT,...,PARKING,DISTRICT,LATITUDE,LONGITUDE,SUBTYPE,DESCRIPTION,rooms,half_rooms,PRICE_PER_METER,TRAM_NEARBY
3,23662566,50,22.9,2,4,"3. kerület, Bécsi út",felújítandó,1950-1980 között,10,van,...,"utca, közterület - fizetős övezet",3,47.532673,19.036491,Eladó panel lakás,"Óbuda lakótelepen, a Bécsi úton 2 szobás laká...",2,0,0.458,True
4,23688806,32,12.5,1,3,"9. kerület, Haller utca",jó állapotú,NaN,3,nincs,...,"utca, közterület",9,47.476099,19.077198,Tégla építésű lakás bérleti joga átadó,"Budapest , IX. kerületi 32 nm-es ,galériázott...",1,0,0.390625,True
21,24171833,85,106.29,1 + 1 fél,0,"2. kerület, Budakeszi út",újszerű,2001-2010 között,6,van,...,teremgarázs hely - benne van az árban,2,47.517581,18.995488,Eladó tégla építésű lakás,Budapest II. ker. Hárshegyen a luxuskivitelű ...,1,1,1.25047,True
25,24278498,96,68.9,3,5,"14. kerület, Angol utca",újszerű,2019,5,van,...,teremgarázs hely - megvásárolható,14,47.516558,19.113785,Eladó tégla építésű lakás,"A 14. kerület egyik legközkedveltebb részén, ...",3,0,0.717708,True
26,24278668,95,68.9,3,4,"14. kerület, Angol utca",újszerű,2019,5,van,...,teremgarázs hely - megvásárolható,14,47.516558,19.113785,Eladó tégla építésű lakás,"A 14. kerület egyik legközkedveltebb részén, ...",3,0,0.725263,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18529,31369438,35,19.9,1,0,"10. kerület, Maglódi út",nincs megadva,NaN,nincs megadva,nincs megadva,...,nincs megadva,10,47.477396,19.165698,Eladó tégla építésű lakás,"Tulajdonostól eladó egy 35m2-es, napfényes, 3...",1,0,0.568571,True
18530,31369447,49,28.5,2,8,"3. kerület, Szőlő utca 74.",felújítandó,NaN,10,van,...,"utca, közterület",3,47.540779,19.037482,Eladó panel lakás,Magánszemély tulajdonostól eladó a közkedvelt...,2,0,0.581633,True
18543,31369957,40,25.5,1 + 1 fél,1,"13. kerület, Béke utca 128.",jó állapotú,1950-1980 között,3,nincs,...,"utca, közterület - fizetős övezet",13,47.546117,19.088497,Eladó tégla építésű lakás,Családi ház vásárlás miatt tulajdonostól elad...,1,1,0.6375,True
18549,31370134,36,27.1,1,4,"9. kerület, Haller utca",jó állapotú,NaN,10,van,...,"utca, közterület - fizetős övezet",9,47.476099,19.077198,Eladó panel lakás,Ferencváros kedvelt részén eladó egy 36 nm-es...,1,0,0.752778,True


In [114]:
df_work['PRICE_PER_METER'] = df_work['PRICE_PER_METER'].astype('float')

In [121]:
df_work[df_work['SUBTYPE'] == 'Eladó tégla építésű lakás'].groupby(['DISTRICT', 'TRAM_NEARBY']).mean().reset_index()

,DISTRICT,TRAM_NEARBY,SIZE,FLOOR,METRO_NEARBY,LATITUDE,LONGITUDE,rooms,PRICE_PER_METER
0,1,False,81.072000,1.512000,0.480000,47.500253,19.033609,2.416000,1.183310
1,1,True,78.580645,1.639785,0.424731,47.498675,19.034010,2.387097,1.018979
2,10,False,62.558621,1.437931,0.006897,47.478353,19.146533,2.113793,0.602992
3,10,True,53.394558,1.517007,0.000000,47.483788,19.129447,1.829932,0.639240
4,11,False,82.845706,1.433770,0.043668,47.465503,19.013122,2.732169,1.012636
5,11,True,68.178723,2.234043,0.397872,47.470226,19.041637,2.193617,0.856928
6,12,False,88.144404,1.010830,0.122744,47.500778,18.998552,2.628159,1.132902
7,12,True,72.940476,1.416667,0.230159,47.495009,19.017832,2.293651,1.023450
8,13,False,66.489381,2.370268,0.578947,47.530405,19.068101,2.261311,0.912620
9,13,True,67.216995,2.535660,0.271624,47.534657,19.077200,2.479514,0.876291


In [115]:
df_district = df_work.groupby(['DISTRICT']).mean().reset_index()

In [116]:
df_district['DISTRICT'] = df_district['DISTRICT'].str.zfill(3)

In [117]:
df_district

,DISTRICT,SIZE,FLOOR,METRO_NEARBY,LATITUDE,LONGITUDE,rooms,PRICE_PER_METER,TRAM_NEARBY
0,001,81.196481,1.642229,0.419355,47.499336,19.033896,2.472141,1.085583,0.563050
1,010,58.378682,2.136045,0.005610,47.480347,19.143771,2.019635,0.595186,0.361851
2,011,70.960000,2.452121,0.187273,47.465549,19.024245,2.395758,0.914148,0.373333
3,012,81.021700,1.218807,0.177215,47.498139,19.007367,2.477396,1.101311,0.475588
4,013,64.712615,2.650376,0.551796,47.535467,19.059412,2.269424,0.893450,0.365079
5,014,62.992612,1.994627,0.108798,47.516051,19.117682,2.122230,0.724830,0.180658
6,015,55.525381,3.017766,0.000000,47.547625,19.133986,1.862944,0.569562,0.441624
7,016,69.768061,1.326996,0.000000,47.517585,19.189080,2.342205,0.675869,0.000000
8,017,68.984962,1.778195,0.000000,47.477034,19.259608,2.413534,0.594475,0.000000
9,018,58.310096,2.512019,0.000000,47.435855,19.175882,1.788462,0.564236,0.149038


### from https://github.com/integralvision/geo-data-hungary/tree/master/GeoJSON

In [39]:
#geojson.GeoJSON(open('./data/GeoJSON/l40-district/001-budapest-01-kerulet.geojson').read())
geojson = folium.GeoJson(data=(open("./data/budapest.geojson", "r")).read())

In [40]:
json = geojson.data

In [43]:

# create a map
m = folium.Map(location=[np.median(df_work['LATITUDE'].tolist()),
                         np.median(df_work['LONGITUDE'].tolist())],
               tiles='Stamen Toner', zoom_start=12)

## add chloropleth layer
m.choropleth(
    geo_data=json,
    name='Average price',
    data=df_district,
    columns=['DISTRICT', 'PRICE_PER_METER'],
    fill_color='YlGn',
    key_on='feature.properties.ksh',
    legend_name='inc_tax_per_capita'
)  

# add markers with basic information
'''
fg = folium.FeatureGroup(name='District Info')
for lat, lon, val, name in zip(gdf_income_tax['centroid_lat'].tolist(),
                               gdf_income_tax['centroid_lon'].tolist(),
                               gdf_income_tax['inc_tax_per_capita'].tolist(),
                               gdf_income_tax['name'].tolist()):
    html = f"""
    <h2>{name}<\h2><br>
    <h4>Income Tax Per Capita: {int(round(val,0))} HUF <\h4>
    """
    fg.add_child(folium.Marker(location=[lat, lon], popup=html))

m.add_child(fg)

# enable layers to be turned in or out
'''

C:\Developer\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


'\nfg = folium.FeatureGroup(name=\'District Info\')\nfor lat, lon, val, name in zip(gdf_income_tax[\'centroid_lat\'].tolist(),\n                               gdf_income_tax[\'centroid_lon\'].tolist(),\n                               gdf_income_tax[\'inc_tax_per_capita\'].tolist(),\n                               gdf_income_tax[\'name\'].tolist()):\n    html = f"""\n    <h2>{name}<\\h2><br>\n    <h4>Income Tax Per Capita: {int(round(val,0))} HUF <\\h4>\n    """\n    fg.add_child(folium.Marker(location=[lat, lon], popup=html))\n\nm.add_child(fg)\n\n# enable layers to be turned in or out\n'

In [44]:
folium.LayerControl().add_to(m)

In [45]:
type(m)

folium.folium.Map

In [46]:
display(m)

In [ ]:
1